In [1]:
# =========================
# CS-340 Project Two Dashboard (Grazioso Salvare)
# =========================

from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import numpy as np
import pandas as pd

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "omor123"

db = AnimalShelter(username, password)

# Unfiltered on load
df = pd.DataFrame.from_records(db.read({}))
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

needed_cols = [
    "breed", "animal_type", "sex_upon_outcome", "age_upon_outcome_in_weeks",
    "location_lat", "location_long", "name", "outcome_type"
]
for c in needed_cols:
    if c not in df.columns:
        df[c] = np.nan

df["location_lat"]  = pd.to_numeric(df["location_lat"],  errors="coerce")
df["location_long"] = pd.to_numeric(df["location_long"], errors="coerce")

RESCUE_QUERIES = {
    "Reset": {},
    "Water Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
    },
    "Mountain or Wilderness Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
    },
    "Disaster or Individual Tracking": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
    },
}

def fetch_df(filter_name="Reset"):
    q = RESCUE_QUERIES.get(filter_name, {})
    dfx = pd.DataFrame.from_records(db.read(q))
    if "_id" in dfx.columns:
        dfx.drop(columns=["_id"], inplace=True, errors="ignore")
    for c in needed_cols:
        if c not in dfx.columns:
            dfx[c] = np.nan
    dfx["location_lat"]  = pd.to_numeric(dfx["location_lat"],  errors="coerce")
    dfx["location_long"] = pd.to_numeric(dfx["location_long"], errors="coerce")
    return dfx

def make_markers(dfx: pd.DataFrame):
    coords = dfx.dropna(subset=["location_lat","location_long"])
    if coords.empty:
        return None
    markers = []
    for _, r in coords.iterrows():
        popup = f"{r.get('name','Unknown')} — {r.get('breed','')}"
        markers.append(
            dl.Marker(
                position=[float(r["location_lat"]), float(r["location_long"])],
                children=[dl.Tooltip(popup),
                          dl.Popup([html.H4("Animal Name"), html.P(str(r.get('name','Unknown')))])]
            )
        )
    return dl.MarkerClusterGroup(children=markers)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Branding: logo + identifier (shown before each section)
image_filename = "Grazioso Salvare Logo.png"  # ensure file exists
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
    LOGO = html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height": "64px", "marginRight": "12px"})
except Exception:
    LOGO = html.Div("Logo missing", style={"color": "crimson", "fontWeight": "bold", "marginRight": "12px"})

def brand_row(title_text, with_link=False):
    right = []
    if with_link:
        # anchor gets its href from dcc.Location via callback
        right = [html.A("Click here to Open Dashboard in a new tab ↗", id="open-new-tab", href="#", target="_blank",
                        style={"marginLeft": "auto", "fontWeight": "600"})]
    return html.Div(
        [LOGO,
         html.Div([html.H2(title_text, style={"margin": 0}),
                   html.Div("Shah Ali Omor • SNHU ID: 2970895", style={"fontStyle": "italic"})])] + right,
        style={"display": "flex", "alignItems": "center", "gap": "12px", "marginBottom": "8px"}
    )

def initial_map(dataframe: pd.DataFrame):
    cluster = make_markers(dataframe)
    children = [dl.TileLayer()]
    if cluster is not None:
        children.append(cluster)
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.27,-97.74], zoom=10, children=children)]

# dcc.Location lets us read the current URL to build a working "open in new tab" link
app.layout = html.Div([
    dcc.Location(id="app-url", refresh=False),

    # Top bar with link
    brand_row("Grazioso Salvare — CS-340 Dashboard", with_link=True),
    html.Hr(),

    html.Center(html.B(html.H1("CS-340 Dashboard"))),
    html.Hr(),

    # Filters
    brand_row("Rescue Type Filter"),
    html.Div([
        dcc.RadioItems(
            id="filter-type",
            options=["Reset","Water Rescue","Mountain or Wilderness Rescue","Disaster or Individual Tracking"],
            value="Reset",
            inline=True
        )
    ]),
    html.Hr(),

    # DataTable
    brand_row("Interactive Data Table"),
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        page_size=8,  # slightly smaller for easier full-page screenshots
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"maxHeight": "350px", "overflowY": "auto"},
        style_cell={"textAlign": "left", "fontFamily": "sans-serif", "fontSize": 13},
    ),

    html.Br(),
    html.Hr(),

    # Chart
    brand_row("Breed Distribution Chart"),
    html.Div(id="graph-id"),

    html.Br(),
    html.Hr(),

    # Map (separate section)
    brand_row("Geolocation Map"),
    html.Div(id="map-id", children=initial_map(df))
])

#############################################
# Interaction Between Components / Controller
#############################################

# Keep the "Open Full Page" link pointing at the actual app URL
@app.callback(Output("open-new-tab", "href"), Input("app-url", "href"))
def _set_ext_link(href):
    return href or "#"

# Update table when filter changes
@app.callback(Output("datatable-id","data"), Input("filter-type", "value"))
def update_dashboard(filter_type):
    dfx = fetch_df(filter_type)
    return dfx.to_dict("records")

# BIG, readable donut chart from current table view
@app.callback(Output("graph-id", "children"), Input("datatable-id", "derived_virtual_data"))
def update_graphs(viewData):
    dfx = pd.DataFrame(viewData) if viewData is not None else df.copy()

    if dfx.empty or dfx['breed'].dropna().empty:
        fig = px.pie(values=[1], names=["No data"], hole=0.45, title="Breed Distribution")
    else:
        # summarize: top N breeds + "Other" for readability
        N = 8
        counts = dfx['breed'].fillna('Unknown').value_counts()
        top = counts.nlargest(N)
        if len(counts) > N:
            other_sum = counts.iloc[N:].sum()
            top = pd.concat([top, pd.Series({'Other': other_sum})])

        pie_df = top.reset_index()
        pie_df.columns = ['breed', 'count']

        fig = px.pie(pie_df, names='breed', values='count', hole=0.45, title="Breed Distribution")

    fig.update_layout(
        height=520,
        margin=dict(l=40, r=40, t=60, b=40),
        legend=dict(orientation="h", y=-0.15),
        font=dict(size=14),
        template="plotly_white"
    )

    return [dcc.Graph(figure=fig, style={"height": "540px", "width": "100%"})]

# Highlight selected columns
@app.callback(Output("datatable-id", "style_data_conditional"), Input("datatable-id", "selected_columns"))
def update_styles(selected_columns):
    return [{"if": {"column_id": i}, "background_color": "#D2F3FF"} for i in (selected_columns or [])]

# Update map based on table data and selected row
@app.callback(Output("map-id", "children"),
              Input("datatable-id", "derived_virtual_data"),
              Input("datatable-id", "derived_virtual_selected_rows"))
def update_map(viewData, index):
    dfx = pd.DataFrame(viewData) if viewData is not None else df.copy()
    if dfx.empty:
        return initial_map(dfx)

    row = 0 if not index else index[0]
    lat = pd.to_numeric(dfx.loc[row, "location_lat"], errors="coerce")
    lon = pd.to_numeric(dfx.loc[row, "location_long"], errors="coerce")

    if np.isnan(lat) or np.isnan(lon):
        return initial_map(dfx)

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[float(lat), float(lon)], zoom=12, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[float(lat), float(lon)], children=[
            dl.Tooltip(str(dfx.loc[row, 'breed'])),
            dl.Popup([html.H4("Animal Name"), html.P(str(dfx.loc[row, 'name']))])
        ])
    ])]

# Inline render + external link available at the top
app.run_server(mode="inline", debug=False, dev_tools_silence_routes_logging=True)
